# 07-5. Encoder Decoder

In [1]:
# RNN 언어 모델

text_data = """옛날 옛적 어느 곳에,
할아버지와 할머니가 살았습니다.
할아버지는 산으로 풀을 베러 갔고,
할머니는 강으로 빨래를 하러 갔습니다.
"""

# text_data 문자열을 set(집합)을 사용하여 중복 제거 & sorted로 정렬
char_to_int = {char : i for i, char in enumerate(sorted(set(text_data)))}
int_to_char = {i:char for char, i in char_to_int.items()}

encoded = [char_to_int[c] for c in text_data]
decoded = [int_to_char[c] for c in encoded]

print(encoded)
print(decoded)

print(''.join(decoded))

[29, 9, 1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0, 37, 25, 19, 34, 30, 1, 37, 18, 12, 4, 1, 23, 26, 24, 12, 13, 3, 0, 37, 25, 19, 34, 11, 1, 22, 31, 16, 1, 35, 32, 1, 20, 15, 1, 5, 7, 2, 0, 37, 18, 12, 11, 1, 6, 31, 16, 1, 21, 14, 17, 1, 36, 15, 1, 5, 24, 12, 13, 3, 0]
['옛', '날', ' ', '옛', '적', ' ', '어', '느', ' ', '곳', '에', ',', '\n', '할', '아', '버', '지', '와', ' ', '할', '머', '니', '가', ' ', '살', '았', '습', '니', '다', '.', '\n', '할', '아', '버', '지', '는', ' ', '산', '으', '로', ' ', '풀', '을', ' ', '베', '러', ' ', '갔', '고', ',', '\n', '할', '머', '니', '는', ' ', '강', '으', '로', ' ', '빨', '래', '를', ' ', '하', '러', ' ', '갔', '습', '니', '다', '.', '\n']
옛날 옛적 어느 곳에,
할아버지와 할머니가 살았습니다.
할아버지는 산으로 풀을 베러 갔고,
할머니는 강으로 빨래를 하러 갔습니다.



In [2]:
print(len(encoded))
print(len(decoded))

73
73


In [4]:
# 제일 짧은 문장(첫 번째 문장)을 기준으로 한 글자 수 (공백 포함)
seq_length = len(text_data.split("\n")[0])
seq_length

12

In [5]:
encoded_text = encoded

sequences = []
# 첫 문장의 글자 수를 빼고 나머지 글자 수만큼 반복
for i in range(len(encoded_text) - seq_length):
    # 인덱스의 끝에 +1을 하여 마지막 글자까지 읽어오게 함
    sequences.append(encoded_text[i:i+seq_length+1])

print(f"시퀀스 길이:{seq_length}")
print(f"레코드 수:{len(sequences)}")
print(f"샘플(처음 3건):\n{sequences[:3]}")

시퀀스 길이:12
레코드 수:61
샘플(처음 3건):
[[29, 9, 1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0], [9, 1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0, 37], [1, 29, 33, 1, 27, 10, 1, 8, 28, 2, 0, 37, 25]]


In [6]:
import numpy as np

# 입력 데이터와 타깃 데이터 생성
X = np.array([seq[:-1] for seq in sequences]) # 신경망 모델에서 입력 데이터의 길이는 항상 고정 >> sequence 길이 >> 12개
y = np.array([seq[1:] for seq in sequences])  # 문장들이 모두 같은 길이라고 할 때, 다음 글자 예측

# e.g. '옛날 옛적 어느 곳에,' 문장
# X: '옛날 옛적 어느 곳에'
# y: '날 옛적 어느 곳에,'

print(X.shape)
print(y.shape)
print(X[:5])
print(y[:5])

(61, 12)
(61, 12)
[[29  9  1 29 33  1 27 10  1  8 28  2]
 [ 9  1 29 33  1 27 10  1  8 28  2  0]
 [ 1 29 33  1 27 10  1  8 28  2  0 37]
 [29 33  1 27 10  1  8 28  2  0 37 25]
 [33  1 27 10  1  8 28  2  0 37 25 19]]
[[ 9  1 29 33  1 27 10  1  8 28  2  0]
 [ 1 29 33  1 27 10  1  8 28  2  0 37]
 [29 33  1 27 10  1  8 28  2  0 37 25]
 [33  1 27 10  1  8 28  2  0 37 25 19]
 [ 1 27 10  1  8 28  2  0 37 25 19 34]]


In [7]:
import tensorflow as tf

# 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(char_to_int), 64, input_length=seq_length),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(char_to_int), activation='softmax')
])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(X, y, epochs=100, batch_size=64, verbose=2)

Epoch 1/100
1/1 - 3s - 3s/step - accuracy: 0.0301 - loss: 3.6368
Epoch 2/100
1/1 - 0s - 75ms/step - accuracy: 0.1639 - loss: 3.6289
Epoch 3/100
1/1 - 0s - 117ms/step - accuracy: 0.2295 - loss: 3.6207
Epoch 4/100
1/1 - 0s - 56ms/step - accuracy: 0.2350 - loss: 3.6120
Epoch 5/100
1/1 - 0s - 75ms/step - accuracy: 0.2363 - loss: 3.6024
Epoch 6/100
1/1 - 0s - 50ms/step - accuracy: 0.2377 - loss: 3.5916
Epoch 7/100
1/1 - 0s - 51ms/step - accuracy: 0.2363 - loss: 3.5795
Epoch 8/100
1/1 - 0s - 47ms/step - accuracy: 0.2336 - loss: 3.5654
Epoch 9/100
1/1 - 0s - 56ms/step - accuracy: 0.2281 - loss: 3.5492
Epoch 10/100
1/1 - 0s - 42ms/step - accuracy: 0.2254 - loss: 3.5302
Epoch 11/100
1/1 - 0s - 59ms/step - accuracy: 0.2186 - loss: 3.5077
Epoch 12/100
1/1 - 0s - 56ms/step - accuracy: 0.2104 - loss: 3.4812
Epoch 13/100
1/1 - 0s - 46ms/step - accuracy: 0.2090 - loss: 3.4498
Epoch 14/100
1/1 - 0s - 52ms/step - accuracy: 0.2049 - loss: 3.4135
Epoch 15/100
1/1 - 0s - 42ms/step - accuracy: 0.2022 - los

In [9]:
# 생성하고자 하는 텍스트의 초기 시퀀스 지정
initial_text = "옛날 옛적 어느 곳에,"  # 첫 번째 레코드를 첫 문장으로 넣음

# 텍스트 생성
max_output_tokens = 16

for _ in range(max_output_tokens):
  encoded_initial_text = [char_to_int[char] for char in initial_text]
  predicted_char = model.predict(np.array([encoded_initial_text]))[:, -1]
  next_char = int_to_char[np.argmax(predicted_char)]
  initial_text += next_char

print(initial_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
옛날 옛적 어느 곳에,
할아버지와 할머니가 살았습니


In [10]:
# 특정 국가(미국) 입력문을 5회씩 반복해서 입력한 코드

import tensorflow as tf
import numpy as np

# 텍스트 데이터 불러오기
# 참고 각 문자 뒤에 스페이스 들어가면 에러 발생
text_data = """Q.영국의 수도는?
A.런던입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.미국의 수도는?
A.Washington DC입니다.
Q.한국의 수도는?
A.서울입니다.
Q.아메리카의 수도는?
A.워싱턴 DC입니다。
Q.미국의 수도는?
A.Washington DC입니다.
Q.잉글랜드의 수도는?
A.런던입니다.
"""

# text_data 문자열을 set(집합)을 사용하여 중복 제거 & sorted로 정렬
char_to_int = {char : i for i, char in enumerate(sorted(set(text_data)))}
int_to_char = {i:char for char, i in char_to_int.items()}

# 텍스트 데이터를 정수열로 변환
encoded = [char_to_int[c] for c in text_data]
# 정수열 데이터를 텍스트 데이터로 변환
decoded = [int_to_char[c] for c in encoded]

# 입력 시퀀스와 타킷 시퀀스 만들기
seq_length = len(text_data.split("\n")[0])

encoded_text = encoded

sequences = []
for i in range(len(encoded_text) - seq_length):
    sequences.append(encoded_text[i:i+seq_length+1])

# 입력 데이터와 타깃 데이터 생성
X = np.array([seq[:-1] for seq in sequences])
y = np.array([seq[1:] for seq in sequences])


# 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(char_to_int), 64, input_length=seq_length),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(char_to_int), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(X, y, epochs=100, batch_size=64, verbose=2)

# 생성할 텍스트의 초기 시퀀스를 지정
initial_text = "Q.미국의 수도는?"

# 텍스트 생성
max_output_tokens = 16

for _ in range(max_output_tokens):
  encoded_initial_text = [char_to_int[char] for char in initial_text]
  predicted_char = model.predict(np.array([encoded_initial_text]))[:, -1]

  next_char = int_to_char[np.argmax(predicted_char)]
  initial_text += next_char

print(initial_text)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


4/4 - 2s - 497ms/step - accuracy: 0.1202 - loss: 3.7768
Epoch 2/100
4/4 - 0s - 33ms/step - accuracy: 0.3339 - loss: 3.7465
Epoch 3/100
4/4 - 0s - 34ms/step - accuracy: 0.3455 - loss: 3.7041
Epoch 4/100
4/4 - 0s - 35ms/step - accuracy: 0.2979 - loss: 3.6374
Epoch 5/100
4/4 - 0s - 34ms/step - accuracy: 0.2511 - loss: 3.5225
Epoch 6/100
4/4 - 0s - 35ms/step - accuracy: 0.1974 - loss: 3.3728
Epoch 7/100
4/4 - 0s - 36ms/step - accuracy: 0.2266 - loss: 3.2243
Epoch 8/100
4/4 - 0s - 27ms/step - accuracy: 0.3129 - loss: 3.0236
Epoch 9/100
4/4 - 0s - 32ms/step - accuracy: 0.4017 - loss: 2.8127
Epoch 10/100
4/4 - 0s - 23ms/step - accuracy: 0.4086 - loss: 2.5942
Epoch 11/100
4/4 - 0s - 22ms/step - accuracy: 0.4322 - loss: 2.3825
Epoch 12/100
4/4 - 0s - 34ms/step - accuracy: 0.4936 - loss: 2.1987
Epoch 13/100
4/4 - 0s - 27ms/step - accuracy: 0.5416 - loss: 2.0308
Epoch 14/100
4/4 - 0s - 32ms/step - accuracy: 0.5734 - loss: 1.8781
Epoch 15/100
4/4 - 0s - 36ms/step - accuracy: 0.6305 - loss: 1.7335
